# Cloud Vision API test

In [ ]:
import os
import io
import dotenv
from pprint import pprint

image_path = '../images/001.jpg'
#image_path = '/home/tombo/Pictures/test.png'

# Load env
dotenv.load_dotenv('../.env')
#print('Credendtials from environ: {}'.format(
#    os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')))

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

# Instantiates a client
client = vision.ImageAnnotatorClient()

# The name of the image file to annotate
#file_name = os.path.join(
#    os.path.dirname(__file__),
#    'resources/wakeupcat.jpg')

# Loads the image into memory
with io.open(image_path, 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Performs label detection on the image file
response = client.label_detection(image=image)
labels = response.label_annotations

print('Labels:')
for label in labels:
    print(label.description)

pprint(response)


In [ ]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')
#
    for text in texts:
        print('\n"{}"'.format(text.description))
#
        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])
#
        print('bounds: {}'.format(','.join(vertices)))
        print(text.bounding_poly.vertices[0],text.bounding_poly.vertices[2])

detect_text(image_path)

In [ ]:
def detect_text_uri(uri):
    """Detects text in the file located in Google Cloud Storage or on the Web.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image()
    image.source.image_uri = uri

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))

#url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/001.jpg'
#url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/015.jpg'
"""
Result:

8172 Miramar Road,
MARGIE SHOOP
San Diego, CA 12793
Manager

Ideal:
住所と役職をまとめて欲しかった。左右方向に走査してるから仕方ないといえば仕方ない。
8172 Miramar Road,
San Diego, CA 12793
MARGIE SHOOP
Manager
"""

#url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/016.jpg'
"""
Result:

Cambridge Logic Ltd
lan Goldsmith
Hardware Engineer
Address: Queen's Road 125, CB3 4AU,
Cambridge, UK
Phone: +44 5858 269466
E-Mail:ian@cambridgelogic.co.uk
"""

#url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/017.jpg'
"""
www.circade.com
CIRCADE
Bernard Desarnauts Founder & CEO
Ph. +61 802 4567 | Fax +61 802 4568
bdesarnauts@circade.com
"""

#url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/026.jpg'
"""
Quest
Specializing in outplacement and resume writing services
8749 Kilbirnie Terrace
Brooklyn Pack, MN 55443
Phone: 541-513-3755 | Fax: 541-513-3876
www.sekelassociates.com
"""

#url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/089.jpg'
"""
Result:
CULTURE
otganicfresen yogurt
www.culturefrozenyogurt.com

Ideal:
CULTURE
organic frozen yogurt(筆記体だったので難しかったのかも)
www.culturefrozenyogurt.com
"""

#url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/090.jpg'
"""
Result:
経済産業省 特許庁
特許審査第二部一般機械
125m
th
ANNIVERSARY
産業財産権制度125周年
審査官
產财梅制シンボルマーク
山本 健晴
T100-8915
東京都千代田区霞が関三丁目4番3号
携帯:650-468-7208 (米国)
080-3450-4412 (日本)
E-mai yamamoto-takeharu@jpo. go. jp
yamatake2525@gmail.com

Ideal:
やっぱり走査が横だから、縦にまとまっている情報が複数列あると情報がバラける
今回なら役職が
経済産業省 特許庁
特許審査第二部一般機械
審査官
のはずなのに、間にロゴの文字の125thとか、産業財産権制度125周年、とかが入り込んできた。
このあたりが課題なのかもしれない。
"""

#url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/096.jpg'
"""
Result:
SUN
SAT
AM
PM
FRI
THURS
WED
TUES
MON
AT
DATE
AVE.
W-
ECHNICIAN NAME:
Apr Oct
Nov-March
UNIVERSITY
NAIL SPA
HOURSH0 URS
Tues Sat 10am-7pm|| Mon-Sat 10am-7p
Sun 10am-5pm
Closed on Monday
(101
NAIL SPA
EMBARCADERO RD,
RE
Sun 10am-50STADIUM
MIDDLEFIELD RD
ALMA ST
650.322.8882| 650.322.8882
LajolieNailSpa.comLajolieNailSpa.com
ASH ST
UNABLE TO KEEP THE APPOINTMENT, A COURTESY OF 24-HOUR ADVANCE NOTICE WILL BE APPRECIATED
EL CAMINO R
ORNIA AV
OREGON EXPY

地図などの図が入ってくるとカオス
枠などで囲ってある情報はひとまとまりにして、出力してほしいけど、グループ化の概念は無いみたい。
隣接する行なら若干、下も読みに行ってるみたいだけど、これでは不十分

"""

#url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/079.jpg'
"""
d.
Receppaşa
Lamartin Cd
Cumhuriyet
MUSAFIR
Şişli
INDIAN
Havaş
Aydede Cd.
$1
Bankasi
Taksim
Gezi
Park
Tarlabaşı Cd.
Taksim
istiklal
Receppaşa Cd. No:7/C
Next to
Talimhane Taksim / ISTANBUL
Beşiktaş
Crystal hotel
Tel:(+90) 212 235 27 41
Opening Hours:
Open everyday:11:00- 23:00
Sundays: 14:00-23:00
www.musafirindian.com
musafir@istanbul.com

向きが90度左になっていたので、ところどころ読み取りミスが出てる
それでもなんとか読めてしまっているからすごい
"""

#url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/071.jpg'
"""
Mark Sm
Fast Repair
2127 El Camino
Palo Alto,CA 94
Fast Repair
CELLULAR REPAIR CENTER
Office:650-322-
Cell :650-492-8
Email:Mark@fastrepair.
"""

url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/003.jpg'
"""
波浪
割烹
NAMI NAMI
Kappo Dining
Shoichi Shiono
executive chef
240 Castro St.
Mountain View, CA 94041
Tel.650-964-6990
"""
detect_text_uri(url)

# Natural Language API test

In [ ]:
import os
import io
import dotenv
from pprint import pprint

# Load env
dotenv.load_dotenv('../.env')

# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze
text = u'Hello, world!'
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))

In [ ]:
import six
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

text = 'President Kennedy spoke at the White House.'
text = """
波浪
割烹
NAMI NAMI
Kappo Dining
Shoichi Shiono
executive chef
240 Castro St.
Mountain View, CA 94041
Tel.650-964-6990
"""

text = """
経済産業省 特許庁
特許審査第二部一般機械
125m
th
ANNIVERSARY
産業財産権制度125周年
審査官
產财梅制シンボルマーク
山本 健晴
T100-8915
東京都千代田区霞が関三丁目4番3号
携帯:650-468-7208 (米国)
080-3450-4412 (日本)
E-mai yamamoto-takeharu@jpo. go. jp
yamatake2525@gmail.com
"""

client = language.LanguageServiceClient()

if isinstance(text, six.binary_type):
    text = text.decode('utf-8')

# Instantiates a plain text document.
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects entities in the document. You can also analyze HTML with:
#   document.type == enums.Document.Type.HTML
entities = client.analyze_entities(document).entities

for entity in entities:
    entity_type = enums.Entity.Type(entity.type)
    print('=' * 20)
    print(u'{:<16}: {}'.format('name', entity.name))
    print(u'{:<16}: {}'.format('type', entity_type.name))
    print(u'{:<16}: {}'.format('salience', entity.salience))
    print(u'{:<16}: {}'.format('wikipedia_url',
          entity.metadata.get('wikipedia_url', '-')))
    print(u'{:<16}: {}'.format('mid', entity.metadata.get('mid', '-')))

    
    
    

In [ ]:
from pprint import pprint
import six
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

required_entities = {'ORGANIZATION':'','PERSON':'','LOCATION':'','ADDRESS':'','PHONE_NUMBER':'',}

def extract_entities(text,required_entities):
    client = language.LanguageServiceClient()
    
    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')
    
    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)
    
    # Detects entities in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    entities = client.analyze_entities(document).entities
    
    for entity in entities:
        entity_type = enums.Entity.Type(entity.type)
        if entity_type.name in required_entities:
            #print(entity.name)
            required_entities[entity_type.name] += entity.name
        #print('=' * 20)
        #print(u'{:<16}: {}'.format('name', entity.name))
        #print(u'{:<16}: {}'.format('type', entity_type.name))
        #print(u'{:<16}: {}'.format('salience', entity.salience))
        #print(u'{:<16}: {}'.format('wikipedia_url',
        #      entity.metadata.get('wikipedia_url', '-')))
        #print(u'{:<16}: {}'.format('mid', entity.metadata.get('mid', '-')))
    pprint(required_entities)
extract_entities(text,required_entities)

# opencv overlay test

In [ ]:
import cv2

image_path = '../images/001.jpg'
alpha = 0.3
image = cv2.imread(image_path)
overlay = image.copy()
output = image.copy()

cv2.rectangle(overlay,(10,20),(30,40),(0,255,0),-1)
cv2.rectangle(overlay,(10,20),(70,79),(0,255,0),-1)
cv2.addWeighted(overlay, alpha, output, 1 - alpha,0, output)
cv2.imwrite('../images/overlay_test.jpg',output)
cv2.imwrite('../images/overlay.jpg',overlay)
#cv2.imshow('output',output)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# json test